# plan

refactor the current 

## custom instruction
what to know: 

- **Problem Context**: The project's goal is to develop a product allocation algorithm. specifically, allocating one type of product from multiple warehouses to multiple orders.  The input to this algorithm consists of the transport time matrix X between warehouses and orders (per unit), an array Y specifying the demand of each order, and an array Z detailing the current stock of the product in each warehouse. The output should be a matrix A indicating the volume of product transported from each warehouse to each order, and the minimize the maximum transportation time, i.e. elements in  Hadamard product fo A and X.

- **Known Constraints**: Sum of allocations for each order≥Demand, Sum of allocations for each warehouse≤Inventory; the objective is to minimize the maximum transportation time across all warehouse-order allocation pairs.
- **Technical Environment**: operate within a modified Jupyter Notebook environment.Python Version: 3.8
- - **Prior Attempts or Solutions**: i've already implement the algorithm using genetic algorithm which i will provide the code for it in the direct prompt below.

how to response
the main goal is to transfer the current project into a ipynb demo, making it easy for my teacher understand my algorithm.
specifically,

if i just provide the function code, please add some explanation comment to help other understand the code easily. after that please give the markdown block that best fit to put above the code block before.

if i ask for adaption or improvement , please  give some advice on how could i improve the code. than give the updated code , denote where you change the code.

note that don't print redundence or niceties.


### Function: json_to_matrices

**Objective**: 

Converts JSON data related to warehouses and orders into numpy matrices. These matrices represent transport times, demand, and inventory levels for multiple goods across multiple orders and warehouses. Additionally, priorities for both orders and warehouses are computed.

**Input**: 

- `json_data`: The JSON data containing information about the orders (`spdd`), warehouse stocks (`ck`), and other associated data.

**Output**: 

- `A1`: A 3D numpy array representing the time needed to deliver a good from each warehouse to each order.
- `A2`: A 2D numpy array representing the demand of each order for each good.
- `A3`: A 2D numpy array representing the current stock of each good in each warehouse.
- `W1`: A 1D numpy array representing the priority of each order.
- `W2`: A 1D numpy array representing the priority of each warehouse.
- `order_list`: A list of all orders.
- `warehouse_list`: A list of all warehouses.
- `goods_list`: A list of all unique goods.
- `goods_dict`: A dictionary mapping goods to their unit size.

**Implementation Details**:

- The number of orders `m` is determined based on the length of `spdd`.
- Unique goods and warehouses are identified and their indices mapped.
- Matrices `A1`, `A2`, `A3` are initialized as zeros and then populated based on the JSON data.
- The priority arrays `W1` and `W2` are computed based on fixed intervals, instead of inverse proportionality.


Changes Made:
- Populated goods_dict before extracting its keys.
- Used sets to create unique all_warehouses and all_goods for faster execution.
- Removed redundant list extractions.
- Used dictionary look-up for good_id to speed up the loops.

In [1]:
import numpy as np
import json

def json_to_matrices(json_data):
    m = len(json_data['spdd'])
    goods_dict = {}
    
    # Extract unique warehouses and populate goods_dict (Updated Code)
    all_warehouses = list({data['cknm'] for order in json_data['spdd'] for data in order['ckdata']})
    
    for warehouse in json_data['ck']:  # Populate goods_dict (New Code)
        warehouse_id = list(warehouse.keys())[0]
        for good in warehouse[warehouse_id]:
            goods_dict[good['spnm']] = good['lg']
    
    all_goods_spdd = {order['spnm'] for order in json_data['spdd']}
    all_goods = sorted(list(all_goods_spdd.union(goods_dict.keys())))  # Updated Code
    
    k = len(all_goods)
    n = len(all_warehouses)
    
    # Initialize matrices (Original Code)
    A1 = np.zeros((m, n, k))
    A2 = np.zeros((m, k))
    A3 = np.zeros((n, k))
    W1 = np.arange(m, 0, -1) / m * 0.3 + 0.7
    W2 = np.arange(n, 0, -1) / n * 0.3 + 0.7
    
    # Use dictionary for faster good_id look-up (New Code)
    good_id_dict = {good: idx for idx, good in enumerate(all_goods)}
    
    # Populate matrices with optimized loops (Updated Code)
    for i, order in enumerate(json_data['spdd']):
        good_id = good_id_dict[order['spnm']]
        A2[i, good_id] = order['sl']
        for j, ckdata in enumerate(order['ckdata']):
            A1[i, j, good_id] = ckdata['dwyssj']
    
    for warehouse in json_data['ck']:
        warehouse_id = list(warehouse.keys())[0]
        i = all_warehouses.index(warehouse_id)
        for good in warehouse[warehouse_id]:
            good_id = good_id_dict[good['spnm']]  # Updated Code
            A3[i, good_id] = good['sl']
    
    return A1, A2, A3, W1, W2, json_data['spdd'], all_warehouses, all_goods, goods_dict  # Updated Code

with open('data/data_5.txt', 'r') as f:
    json_data = json.load(f)
    A1, A2, A3, W1, W2, order_list, warehouse_list, goods_list, goods_dict = json_to_matrices(json_data)
    print(order_list)
    print(warehouse_list)
    print(goods_list)
    print(goods_dict)


[{'ddnm': 'Order1', 'qynm': 'Company1', 'spnm': 'Good1', 'sl': 7, 'lg': '枚', 'ckdata': [{'cknm': 'cknm1', 'dwyssj': 3.0}, {'cknm': 'cknm2', 'dwyssj': 4.0}]}, {'ddnm': 'Order2', 'qynm': 'Company1', 'spnm': 'Good1', 'sl': 4, 'lg': '枚', 'ckdata': [{'cknm': 'cknm1', 'dwyssj': 2.0}, {'cknm': 'cknm2', 'dwyssj': 1.0}]}, {'ddnm': 'Order3', 'qynm': 'Company1', 'spnm': 'Good1', 'sl': 3, 'lg': '枚', 'ckdata': [{'cknm': 'cknm1', 'dwyssj': 1.0}, {'cknm': 'cknm2', 'dwyssj': 3.0}]}]
['cknm1', 'cknm2']
['Good1']
{'Good1': '枚'}
